In [1]:
import pandas as pd
import numpy as np 

In [2]:
movies=pd.read_csv("tmdb_5000_movies.csv")   #TWO DATASETS,LATER ON MERGED 
credits=pd.read_csv("tmdb_5000_credits.csv") 

In [13]:
# movies.head() #sanity check lol lesgoo

In [ ]:
# credits.head()

In [3]:
#MERGED THE TWO DATASETS

movies=movies.merge(credits,on="title")

In [4]:
#CHECKING COL NAMES TO REMOVE UNECESSARY COLUMNS 
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [17]:
# movies['original_language'].value_counts()                   #imbalanced dataset with 95% being english so not retaining 'language' tag

In [5]:
movies['genres'][0]                                            #format check, get the name of the genre and drop any irrelvant info

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [6]:
movies['keywords'][0]     

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [7]:
movies['overview'][0]      

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [8]:
# movies['cast'][0]     #lengthy output alert!

In [9]:
# movies['crew'][0]     

In [10]:
movies=movies[['id','title','overview','genres','keywords','cast','crew']]   # USEFUL COLUMNS RETAINED

In [11]:
#MISSING DATA

movies.isnull().sum()                    #  in overview 3 columns are null so drop them
movies.dropna(inplace=True)

#duplicate data
movies.duplicated().sum()                #no duplicates so chill

0

# PREPROCESSING 

In [12]:
#   we want the genres data in the form of ex: ['action','adventure','fantasy','scifi']
#   we wish to have keywords, cast(only the top 3 names) , crew (only the director) in a format similar to genres list
#   each of the fields in genre is listed as "string of list of dictionaries" so we have to convert the string of list into an actual list

In [13]:
import ast 
# ast.literal_eval()  #function to convert an string object into list
def convert(obj):
    """
    args: string object that has the genre details of a movie
    output: list of genres

    description:
    This function takes a string object as input, which includes the genre details of a movie.
    It processes this string to extract individual genres and returns them as a list of genres. 
    The genres are typically separated by commas or other delimiters within the string. 
    The function ensures that each genre is correctly identified and included in the output list.
    """
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [14]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]')    #sanity check of the defined function 

['Action', 'Adventure', 'Crime']

In [15]:
#convert all the "string of list of dictionaries" to required format:"list" lesgooo!
movies['genres']=movies['genres'].apply(convert)   

In [16]:
movies['keywords']=movies['keywords'].apply(convert) 


In [17]:
# Extracting only top 3 actors of the movie

In [18]:
def convert3(obj):
    L=[]
    counter =0
    for i in ast.literal_eval(obj):
        if(counter<3):
            L.append(i['name'])
            counter+=1
    return L

In [19]:
movies['cast']=movies['cast'].apply(convert3)

In [20]:
movies.head(1)  #SANITY CHECK 

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [41]:
#CREW YET TO BE PROCESSED,FETCHING ONLY THE DIRECTOR

In [21]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            L.append(i['name'])
    return L


# movies.head()

In [22]:
movies['crew']=movies['crew'].apply(fetch_director)  #PRE-PROCESSING CREW 

In [23]:
movies.head(1) #SANITY CHECK 

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [24]:
movies['overview']=movies['overview'].apply(lambda x: x.split())   #get the words,later combine all words,handle duplicates 

In [25]:
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [26]:
#remove all whitespaces in names, if firstname is same but last name is different (esp. cast or crew),accuracy is compromised in such cases

def word_join(list_of_striing):
    list_of_string=[]
    for i in list_of_striing:
        
        i=i.replace(" ","")
        list_of_string.append(i)
    return list_of_string

movies['cast']=movies['cast'].apply(word_join)
movies['genres']=movies['genres'].apply(word_join)
movies['crew']=movies['crew'].apply(word_join)
movies['keywords']=movies['keywords'].apply(word_join)
    
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [27]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew'] # merge columns to form a single column called "tags" which we will be using ultimately to recommend movies

In [28]:
new_df=movies[['id','title','tags']]  #created a new df,just three coulmns

In [29]:
new_df.head(3) #sanity check

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."


In [30]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))             #concatenate all the tag words to form a huge string which has info related to cast,crew,overview,keywords

/var/folders/9y/k5wbg7tx10g97tt7n41gf9qr0000gn/T/ipykernel_10588/146925741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))             #concatenate all the tag words to form a huge string which has info related to cast,crew,overview,keywords


In [31]:
new_df.head() #sanity check

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [32]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower()) #convert to lower case

/var/folders/9y/k5wbg7tx10g97tt7n41gf9qr0000gn/T/ipykernel_10588/4128951458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower()) #convert to lower case


In [33]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('punkt')           

PS=PorterStemmer()                #very important step,we just have to retain the root word (danced,dancing-->danc) 

                                  # in stemming if ['love','loving','loved'] is the input then on stemming it gives the output ['love','love','love']

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vamshivishruthj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# stemmed_words=[PS.stem(words) for words in feature_names]
#process of reducing is called stemming, define a fctn for the same


def stem(text):
    """
    args: string to be stemmed 
    output: stemmed string
    
    description:
    takes each of the words in the inpput sentence and stems it by reducing the whole word to its root word,
    joins the root words and returns it 
    """
    y=[]

    for i in text.split():
       y.append(PS.stem(i))
    return " ".join(y)

In [35]:
new_df['tags']=new_df['tags'].apply(stem)  # apply to tags

/var/folders/9y/k5wbg7tx10g97tt7n41gf9qr0000gn/T/ipykernel_10588/3376506858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)  # apply to tags


In [36]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer converts the sentence(each tag) into a feature vector
cv=CountVectorizer(max_features=6000,stop_words="english")
vectors=cv.fit_transform(new_df['tags']).toarray()           # by default cv returns a scipy sparse matrix so convert it to  array to handle it better

In [37]:
vectors.shape                        # so for all the 4806 movies we have 4086 vectors each vector has 6000 features

(4806, 6000)

In [38]:
feature_names=cv.get_feature_names_out() # this lists out the features so used to form the vector 

# note that here also there could be lot of errors, few nubers may not be of significance, while numbers like 1930,1940 etc are useful in telling h=which year the movie was released etc, so it is significant
# where as few words like activity and activities both of them have the same meaning but just a samll spelling difference in such a case we want only one word,note that many such singular vs plural repitions
# are there but that's not a issue bcz only the unqique words are used for generating features and such cases have been handled by stemming


In [39]:
feature_names

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [40]:
new_df['tags'][0] #sanity check

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [41]:
stem('in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron')


'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [45]:
from sklearn.metrics.pairwise import cosine_similarity       
#lets use cosine similariy for this proj, each of the tags is now a feature vector in 6000-D space, cosine similarity measures teh angle btw the chosen vetor and the rest of the feature vectors
#cosine similairty over eculidian distance bcz of higher accuracy in cases of high dimensions
similarity_matrix=cosine_similarity(vectors)      #array of arrays, each movie vs all the movies



In [51]:
import pickle 
pickle.dump(similarity_matrix,open("similarity_matrix.pkl","wb"))

In [47]:
type(similarity_matrix)

numpy.ndarray

In [48]:
def recommend(movie):         
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity_matrix[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)
        

In [49]:
recommend("Batman Begins")

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [ ]:
#yay

In [ ]:
#ongoing work on deployment


pickle.dump(new_df,open("movies.pkl","wb"))  

In [ ]:
pickle.dump()